In [1]:
# This code was made with the help of AI tools, including ChatGPT and GitHub Copilot.
import pandas as pd
import requests
from urllib.parse import urlparse
from tqdm import tqdm
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import nibabel as nib
import io
import gzip


In [2]:

# find the latest version of a given OpenNueo datsey
def get_latest_snapshot(dataset_id: str):
    """
    Fetch the latest snapshot tag for a given OpenNeuro dataset ID.

    Parameters
    ----------
    dataset_id : str
        The OpenNeuro dataset ID (e.g., "ds000001").
    
    Returns
    -------
    str
        The latest snapshot tag (version) for the dataset.

    """
    graphql_url = "https://openneuro.org/crn/graphql" # GraphQL endpoint for OpenNeuro
    
    # GraphQL query to fetch dataset snapshots
    query = """
      query ($id: ID!) {
        dataset(id: $id) {
          snapshots {
            tag
          }
        }
      }
    """

    # Make the GraphQL request to fetch snapshots
    res = requests.post(graphql_url, json={"query": query, "variables": {"id": dataset_id}})
    res.raise_for_status() # Check for HTTP errors
    snaps = res.json()["data"]["dataset"]["snapshots"] # Extract snapshots
    if not snaps: # Check if there are any snapshots
        raise Exception("No snapshots found.")
    latest = sorted([s["tag"] for s in snaps], reverse=True)[0] # Get the latest tag
    return latest


In [3]:

# # Get the urls of all JSON files for a specific version of an OpenNeuro dataset
# def get_json_urls(dataset_id: str, version_tag: str):
#     """
#     Fetch URLs of JSON files for a specific version of an OpenNeuro dataset.
    
#     Parameters
#     ----------
#     dataset_id : str
#         The OpenNeuro dataset ID (e.g., "ds000001").
#     version_tag : str
#         The version tag (snapshot) to fetch JSON files from.

#     Returns
#     -------
#     dict
#         A dictionary mapping JSON filenames to their download URLs.
#     """

#     graphql_url = "https://openneuro.org/crn/graphql" # GraphQL endpoint for OpenNeuro

#     # GraphQL query to fetch files for a specific snapshot
#     query = """
#       query ($id: ID!, $tag: String!) {
#         snapshot(datasetId: $id, tag: $tag) {
#             files {
#             filename
#             urls
#             }
#         }
#     }
#     """

#     # Make the GraphQL request to fetch files for the specified snapshot
#     res = requests.post(graphql_url, json={"query": query, "variables": {"id": dataset_id, "tag": version_tag}})
#     if res.status_code != 200: # Check for HTTP errors
#         print("Response content:\n", res.text)
#         raise Exception(f"GraphQL error fetching version {version_tag}: {res.status_code}")
#     files = res.json()["data"]["snapshot"]["files"]
    
#     # Recursively collect all .json files
#     def collect_json_files(directory, current_path=""):
#         jsons = {}
#         for f in directory.get("files", []):
#             if f["filename"].endswith(".json"):
#                 full_path = f"{current_path}{f['filename']}"
#                 jsons[full_path] = f["urls"][0]
#         for subdir in directory.get("directories", []):
#             sub_path = f"{current_path}{subdir['name']}/"
#             jsons.update(collect_json_files(subdir, sub_path))
#         return jsons
    
#     json_files = {f["filename"]: f["urls"][0] for f in files if f["filename"].endswith(".json")}

#     print(f"Found {len(json_files)} JSON files.")
#     print("Examples:")
#     for example in list(json_files.keys())[:5]:
#         print(" --", example)

#     return json_files


In [4]:
# def get_json_urls(dataset_id: str, version_tag: str):
#     """
#     Fetch URLs of all JSON files for a specific version of an OpenNeuro dataset.

#     Parameters
#     ----------
#     dataset_id : str
#         The OpenNeuro dataset ID (e.g., "ds000001").
#     version_tag : str
#         The version tag (snapshot) to fetch JSON files from.

#     Returns
#     -------
#     dict
#         A dictionary mapping JSON file paths (with folders) to their download URLs.
#     """

#     graphql_url = "https://openneuro.org/crn/graphql"

#     query = """
#     query ($id: ID!, $tag: String!) {
#       snapshot(datasetId: $id, tag: $tag) {
#         files {
#           filename
#           urls
#         }
#       }
#     }
#     """

#     res = requests.post(graphql_url, json={"query": query, "variables": {"id": dataset_id, "tag": version_tag}})
#     if res.status_code != 200:
#         print("Response content:\n", res.text)
#         raise Exception(f"GraphQL error fetching version {version_tag}: {res.status_code}")

#     files = res.json()["data"]["snapshot"]["files"]

#     # Filter for JSON files and map full path -> first url
#     json_files = {f["filename"]: f["urls"][0] for f in files if f["filename"].endswith(".json")}

#     print(f"✅ Found {len(json_files)} JSON files.")
#     print("Examples:")
#     for example in list(json_files.keys())[:5]:
#         print("  -", example)

#     return json_files


In [5]:
# import requests

# def get_json_urls(dataset_id: str, version_tag: str):
#     """
#     Fetch all JSON files (including nested) for a given dataset and version using the OpenNeuro REST API manifest.

#     Returns dict mapping full path -> URL
#     """
#     manifest_url = f"https://openneuro.org/crn/datasets/{dataset_id}/snapshots/{version_tag}/contents"

#     res = requests.get(manifest_url)
#     res.raise_for_status()

#     contents = res.json()  # This is a list of file/directory dicts recursively

#     json_files = {}

#     def recurse_files(entries, prefix=""):
#         for entry in entries:
#             if entry["type"] == "file" and entry["name"].endswith(".json"):
#                 path = prefix + entry["name"]
#                 # The URL is in 's3Uri' or 'url' or build from key? Let's check:
#                 # Try 'url' field if present, else construct from s3Uri
#                 url = entry.get("url")
#                 if not url:
#                     url = f"https://s3.amazonaws.com/openneuro.org/{dataset_id}/{path}"
#                 json_files[path] = url
#             elif entry["type"] == "directory":
#                 recurse_files(entry["contents"], prefix + entry["name"] + "/")

#     recurse_files(contents)

#     print(f"✅ Found {len(json_files)} JSON files.")
#     print("Examples:")
#     for example in list(json_files.keys())[:5]:
#         print(" -", example)

#     return json_files


In [6]:
# import boto3

# def list_s3_json_files(dataset_id):
#     s3 = boto3.client('s3')
#     bucket = 'openneuro.org'
#     prefix = f"{dataset_id}/"

#     paginator = s3.get_paginator('list_objects_v2')
#     pages = paginator.paginate(Bucket=bucket, Prefix=prefix)

#     json_files = {}

#     for page in pages:
#         for obj in page.get('Contents', []):
#             key = obj['Key']
#             if key.endswith('.json'):
#                 url = f"https://s3.amazonaws.com/{bucket}/{key}"
#                 json_files[key] = url

#     print(f"Found {len(json_files)} JSON files.")
#     for example in list(json_files.keys())[:5]:
#         print(" -", example)

#     return json_files

# json_urls = list_s3_json_files('ds005264')


In [7]:
def list_s3_json_files(dataset_id):
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    bucket = 'openneuro.org'
    prefix = f"{dataset_id}/"

    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket, Prefix=prefix)

    json_files = {}

    for page in pages:
        for obj in page.get('Contents', []):
            key = obj['Key']
            if key.endswith('.json'):
                url = f"https://s3.amazonaws.com/{bucket}/{key}"
                json_files[key] = url

    print(f"Found {len(json_files)} JSON files.")
    for example in list(json_files.keys())[:10]:
        print(" -", example)

    return json_files


In [8]:

# flatten json objects into a single-level dictionary
def flatten_json(y):
    """
    Flatten a nested JSON object into a single-level dictionary with dot notation keys.

    Parameters
    ----------
    y : dict
        The nested JSON object to flatten.
    
    Returns
    -------
    dict
        A flattened dictionary with keys in dot notation.
    """

    out = {} # Initialize output dictionary

    # Recursive function to flatten the JSON
    def _flatten(x, name=""):
        if isinstance(x, dict):
            for k,v in x.items():
                _flatten(v, name + k + ".")
        elif isinstance(x, list):
            for i,v in enumerate(x):
                _flatten(v, name + str(i) + ".")
        else:
            out[name[:-1]] = x
    _flatten(y)

    return out


In [9]:

# # Load JSON files from an OpenNeuro dataset URL into a Pandas DataFrame
# def jsons_to_dataframe(openneuro_url: str):
#     """
#     Load JSON files from an OpenNeuro dataset URL into a Pandas DataFrame.

#     Parameters
#     ----------
#     openneuro_url : str
#         The OpenNeuro dataset URL (e.g., "https://openneuro.org/datasets/ds000001/versions/1.0.0").
    
#     Returns
#     -------
#     pd.DataFrame
#         A Pandas DataFrame containing the flattened JSON data, indexed by file name.
#     """

#     parsed = urlparse(openneuro_url)
#     parts = parsed.path.strip("/").split("/")
#     dataset_id = parts[1]
#     version = parts[3] if len(parts) > 3 and parts[2] == "versions" else None

#     if version is None:
#         version = get_latest_snapshot(dataset_id) #######
#         print(f"No version specified — using latest: {version}")
#     else:
#         print(f"Using specified version: {version}")

#     json_urls = get_json_urls(dataset_id, version) #######
#     print(f"Found {len(json_urls)} JSON files for version {version}")

#     records = []
#     for fname, url in tqdm(json_urls.items(), desc="Downloading JSONs"):
#         r = requests.get(url)
#         r.raise_for_status()
#         flat = flatten_json(r.json()) #######
#         flat["__file__"] = fname
#         records.append(flat)

#     df = pd.DataFrame(records).set_index("__file__")
#     return df

In [10]:
def jsons_to_dataframe(openneuro_url: str):
    """
    Load JSON files from an OpenNeuro dataset URL into a Pandas DataFrame.

    Parameters
    ----------
    openneuro_url : str
        The OpenNeuro dataset URL (e.g., "https://openneuro.org/datasets/ds000001/versions/1.0.0").
    
    Returns
    -------
    pd.DataFrame
        A Pandas DataFrame containing the flattened JSON data, indexed by file name.
    """

    parsed = urlparse(openneuro_url)
    parts = parsed.path.strip("/").split("/")
    dataset_id = parts[1]
    # version is not used for S3 listing, so no need to parse it here

    print(f"Listing JSON files for dataset: {dataset_id} from S3 (no versioning)")

    # Call your S3 JSON file lister WITHOUT version arg
    json_urls = list_s3_json_files(dataset_id)  # <-- Removed version argument here

    print(f"Found {len(json_urls)} JSON files.")

    records = []
    for fname, url in tqdm(json_urls.items(), desc="Downloading JSONs"):
        r = requests.get(url)
        r.raise_for_status()
        flat = flatten_json(r.json())
        flat["__file__"] = fname
        records.append(flat)

    df = pd.DataFrame(records).set_index("__file__")
    return df


In [11]:
# get_json_urls("ds005264", "1.0.0")

In [12]:
# # Example usage
# json_urls = list_s3_json_files('ds005264')

Found 296 JSON files.
 - ds005264/dataset_description.json
 - ds005264/participants.json
 - ds005264/sub-01/anat/sub-01_acq-1_T1w.json
 - ds005264/sub-01/anat/sub-01_acq-2_T1w.json
 - ds005264/sub-01/anat/sub-01_acq-3_T1w.json
 - ds005264/sub-01/fmap/sub-01_dir-AP_epi.json
 - ds005264/sub-01/fmap/sub-01_dir-PA_epi.json
 - ds005264/sub-01/func/sub-01_task-rest_echo-1_bold.json
 - ds005264/sub-01/func/sub-01_task-rest_echo-1_sbref.json
 - ds005264/sub-01/func/sub-01_task-rest_echo-2_bold.json


In [13]:
# df = jsons_to_dataframe("https://openneuro.org/datasets/ds005264")
# df.head()

Listing JSON files for dataset: ds005264 from S3 (no versioning)
Found 296 JSON files.
 - ds005264/dataset_description.json
 - ds005264/participants.json
 - ds005264/sub-01/anat/sub-01_acq-1_T1w.json
 - ds005264/sub-01/anat/sub-01_acq-2_T1w.json
 - ds005264/sub-01/anat/sub-01_acq-3_T1w.json
 - ds005264/sub-01/fmap/sub-01_dir-AP_epi.json
 - ds005264/sub-01/fmap/sub-01_dir-PA_epi.json
 - ds005264/sub-01/func/sub-01_task-rest_echo-1_bold.json
 - ds005264/sub-01/func/sub-01_task-rest_echo-1_sbref.json
 - ds005264/sub-01/func/sub-01_task-rest_echo-2_bold.json
Found 296 JSON files.


,Name,BIDSVersion,License,Authors.0,Authors.1,Authors.2,Authors.3,Authors.4,Authors.5,Authors.6,...,SliceTiming.90,SliceTiming.91,SliceTiming.92,SliceTiming.93,SliceTiming.94,SliceTiming.95,SliceTiming.96,SliceTiming.97,SliceTiming.98,SliceTiming.99
__file__,,,,,,,,,,,,,,,,,,,,,
ds005264/dataset_description.json,SoCal Kinesia and Incentivization for Parkinso...,1.8.0,CC0,Neil M. Dundon,Elizabeth Rizor,Joanne Stasiak,Jingyi Wang,Kiana Sabugo,Christina Villaneuva,Parker Barandon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ds005264/participants.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ds005264/sub-01/anat/sub-01_acq-1_T1w.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ds005264/sub-01/anat/sub-01_acq-2_T1w.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ds005264/sub-01/anat/sub-01_acq-3_T1w.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# df.index.tolist()

In [15]:


# def get_scan_duration(df):
#     """
#     Calculate the duration of each scan in seconds from combinations of the 'AcquisitionDuration','SeriesTime','AcquisitionTime', and 'RepetitionTime' fields in the DataFrame.

#     Parameters
#     ----------
#     df : pd.DataFrame
#         The DataFrame containing flattened JSON data with a 'duration' field.
    
#     Returns
#     -------
#     pd.Series
#         A Pandas Series containing the scan durations in seconds.
#     """

#     # check if dataframe has any of the required columns
#     required_columns = ['AcquisitionDuration', 'SeriesTime', 'AcquisitionTime', 'RepetitionTime', 'NumberOfVolumes']
#     if not any(col in df.columns for col in required_columns):
#         raise ValueError(f"DataFrame lacks all required columns: {required_columns}")
    

In [16]:
df = jsons_to_dataframe("https://openneuro.org/datasets/ds005118")
df.head()

Listing JSON files for dataset: ds005118 from S3 (no versioning)
Found 240 JSON files.
 - ds005118/dataset_description.json
 - ds005118/sub-ME01/ses-func01/fmap/sub-ME01_ses-func01_dir-AP_run-01_epi.json
 - ds005118/sub-ME01/ses-func01/fmap/sub-ME01_ses-func01_dir-PA_run-01_epi.json
 - ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-01_bold.json
 - ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-02_bold.json
 - ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-03_bold.json
 - ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-04_bold.json
 - ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-05_bold.json
 - ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-02_echo-01_bold.json
 - ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-02_echo-02_bold.json
Found 240 JSON files.


,Name,BIDSVersion,License,Authors.0,Authors.1,HowToAcknowledge,Funding.0,ReferencesAndLinks.0,DatasetDOI,Modality,...,SliceTiming.66,SliceTiming.67,SliceTiming.68,SliceTiming.69,SliceTiming.70,SliceTiming.71,EchoNumber,TaskName,CogAtlasID,Instructions
__file__,,,,,,,,,,,,,,,,,,,,,
ds005118/dataset_description.json,Weill Cornell Medicine Multi-echo (WCM-ME) Dat...,1.0.0,CC0,Charles J. Lynch,Conor Liston,"When using this data, please cite Lynch et al....",,https://pubmed.ncbi.nlm.nih.gov/33357444/,doi:10.18112/openneuro.ds005118.v1.0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ds005118/sub-ME01/ses-func01/fmap/sub-ME01_ses-func01_dir-AP_run-01_epi.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ds005118/sub-ME01/ses-func01/fmap/sub-ME01_ses-func01_dir-PA_run-01_epi.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-01_bold.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR,...,0.6675,1.2225,0.445,1.0,0.2225,0.7775,NaN,NaN,NaN,NaN
ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-02_bold.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR,...,0.6675,1.2225,0.445,1.0,0.2225,0.7775,2.0,NaN,NaN,NaN


In [17]:
# 'AcquisitionDuration', 'SeriesTime', 'AcquisitionTime', 'RepetitionTime', 'NumberOfVolumes'
df.columns.tolist()

['Name',
 'BIDSVersion',
 'License',
 'Authors.0',
 'Authors.1',
 'HowToAcknowledge',
 'Funding.0',
 'ReferencesAndLinks.0',
 'DatasetDOI',
 'Modality',
 'MagneticFieldStrength',
 'Manufacturer',
 'ManufacturersModelName',
 'InstitutionName',
 'InstitutionalDepartmentName',
 'InstitutionAddress',
 'DeviceSerialNumber',
 'StationName',
 'BodyPartExamined',
 'PatientPosition',
 'ProcedureStepDescription',
 'SoftwareVersions',
 'MRAcquisitionType',
 'SeriesDescription',
 'ProtocolName',
 'ScanningSequence',
 'SequenceVariant',
 'ScanOptions',
 'SequenceName',
 'ImageType.0',
 'ImageType.1',
 'ImageType.2',
 'ImageType.3',
 'SeriesNumber',
 'AcquisitionTime',
 'AcquisitionNumber',
 'SliceThickness',
 'SpacingBetweenSlices',
 'SAR',
 'EchoTime',
 'RepetitionTime',
 'FlipAngle',
 'PartialFourier',
 'BaseResolution',
 'ShimSetting.0',
 'ShimSetting.1',
 'ShimSetting.2',
 'ShimSetting.3',
 'ShimSetting.4',
 'ShimSetting.5',
 'ShimSetting.6',
 'ShimSetting.7',
 'TxRefAmp',
 'PhaseResolution',
 

In [18]:
df.tail(1)

,Name,BIDSVersion,License,Authors.0,Authors.1,HowToAcknowledge,Funding.0,ReferencesAndLinks.0,DatasetDOI,Modality,...,SliceTiming.66,SliceTiming.67,SliceTiming.68,SliceTiming.69,SliceTiming.70,SliceTiming.71,EchoNumber,TaskName,CogAtlasID,Instructions
__file__,,,,,,,,,,,,,,,,,,,,,
ds005118/task-rest_bold.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MR,...,0.6675,1.2225,0.445,1.0,0.2225,0.7775,NaN,rest,http://www.cognitiveatlas.org/term/id/trm_4c8a...,All you need to do is simply relax and look at...


In [27]:
def list_s3_niigz_files(dataset_id):
    """
    List all .nii.gz files in a given OpenNeuro dataset (via public S3).

    Parameters
    ----------
    dataset_id : str
        The OpenNeuro dataset ID (e.g., "ds000001").
    
    Returns
    -------
    dict
        A dictionary mapping file paths to their download URLs.
    """
    
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    bucket = 'openneuro.org'
    prefix = f"{dataset_id}/"

    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket, Prefix=prefix)

    niigz_files = {}

    for page in pages:
        for obj in page.get('Contents', []):
            key = obj['Key']
            if key.endswith('.nii.gz'):
                url = f"https://s3.amazonaws.com/{bucket}/{key}"
                niigz_files[key] = url

    print(f"Found {len(niigz_files)} NIfTI files (.nii.gz).")
    for example in list(niigz_files.keys())[:10]:
        print(" -", example)

    return niigz_files


In [37]:
# nii_files = list_s3_niigz_files("ds003814")
# nii_files

In [28]:
def get_nii_shape_from_url(url):
    """
    Fetch a NIfTI file from a URL, decompress it if gzipped, and return its shape.

    Parameters
    ----------
    url : str
        The s3 URL of the NIfTI file (can be gzipped).

    Returns
    -------
    tuple
        The shape of the NIfTI image (dimensions).
    """
    response = requests.get(url)
    response.raise_for_status()

    # Decompress gzip content
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz:
        decompressed = gz.read()

    # Use FileHolder with decompressed bytes for header and image
    file_holder = nib.FileHolder(fileobj=io.BytesIO(decompressed))
    img = nib.Nifti1Image.from_file_map({'header': file_holder, 'image': file_holder})

    return img.shape

In [29]:
url = "https://s3.amazonaws.com/openneuro.org/ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-01_bold.nii.gz"
shape = get_nii_shape_from_url(url)
print("Shape:", shape)

Shape: (90, 90, 72, 640)


In [32]:
shape[3]

640

In [ ]:
# ds005118/sub-ME01/ses-func01/func/sub-ME01_ses-func01_task-rest_run-01_echo-01_bold.json

In [36]:
def get_scan_time_from_TR(json_path: str, TR: float = None):
    """
    Calculate the total scan time in seconds based on TR and number of volumes from a JSON file.

    Parameters
    ----------
    json_path : str
        The path to the JSON file containing scan metadata.
    TR : float
        The repetition time (TR) in seconds.
    
    Returns
    -------
    float
        The total scan time in seconds.
    """
    
    
    if TR is None:
        with open(json_path, 'r') as f:
            metadata = flatten_json(pd.read_json(f).to_dict(orient='records')[0])
            TR = metadata.get('RepetitionTime', None)
            if TR is None:
                raise ValueError("JSON file does not contain 'RepetitionTime' field.")

    num_volumes = metadata.get('NumberOfVolumes', None)
    if num_volumes is None:
        shape = get_nii_shape_from_url(url)
        if len(shape) < 4:
            raise ValueError("NIfTI file does not have a time dimension (4th dimension).")
        num_volumes = shape[3]

    return num_volumes * TR

def convert_hhmmss_string(time_str: str):
    """
    Convert a time string in HH:MM:SS format to seconds.

    Parameters
    ----------
    time_str : str
        The time string in HH:MM:SS format.

    Returns
    -------
    float   
        The time in seconds.
    """
    if not isinstance(time_str, str):
        raise ValueError("Time string must be a string in HH:MM:SS format.")
    
    parts = time_str.split(':')
    if len(parts) != 3:
        raise ValueError("Time string must be in HH:MM:SS format.")

    hours, minutes, seconds = map(float, parts)
    return hours * 3600 + minutes * 60 + seconds

def get_scan_time_from_AcquisitionTime_SeriesTime(json_path: str, AcquisitionTime: str = None, SeriesTime: str = None):
    """
    Calculate the scan duration in seconds based on AcquisitionTime and SeriesTime from a JSON file.

    Parameters
    ----------
    json_path : str
        The path to the JSON file containing scan metadata.
    AcquisitionTime : str
        The acquisition time in HH:MM:SS format.
    SeriesTime : str
        The series time in HH:MM:SS format.
    
    Returns
    -------
    float
        The scan duration in seconds.
    """
    
    with open(json_path, 'r') as f:
        metadata = flatten_json(pd.read_json(f).to_dict(orient='records')[0])
    
    if AcquisitionTime is None:
        AcquisitionTime = metadata.get('AcquisitionTime', None)
        if AcquisitionTime is None:
            raise ValueError("JSON file does not contain 'AcquisitionTime' field.")
    
    if SeriesTime is None:
        SeriesTime = metadata.get('SeriesTime', None)
        if SeriesTime is None:
            raise ValueError("JSON file does not contain 'SeriesTime' field.")
    
    return time_to_seconds(AcquisitionTime) - time_to_seconds(SeriesTime)
    

In [ ]:
# 'AcquisitionDuration', 'SeriesTime', 'AcquisitionTime', 'RepetitionTime', 'NumberOfVolumes'